In [14]:
nb_users_per_sector  = 8;
nb_sectors           = 9;
nb_RBs               = 25;
total_nb_users       = nb_users_per_sector * nb_sectors;
min_power_per_RB     = 0.1;
max_power_per_sector = 20; 
noise        = float(10)^(-20.4);
bw         = 180e3;

In [15]:
using JuMP

In [16]:
pathloss=rand(nb_users_per_sector,nb_sectors,nb_RBs);

In [17]:
m = Model()
@defVar(m, poweralloc[1:nb_sectors,1:nb_RBs] >= min_power_per_RB);

In [18]:
for j in 1:nb_sectors
    for k=1:nb_RBs
        setValue(poweralloc[j,k], 1+rand(1)[1])
    end
end

In [19]:
@setNLObjective(m, Max, sum{log(log(1+(poweralloc[j,k]*pathloss[i,j,k]/(bw*noise+sum{poweralloc[j_p,k]*pathloss[i,j_p,k], j_p=1:nb_sectors; j_p!=j})))), i=1:nb_users_per_sector, j=1:nb_sectors, k=1:nb_RBs})

In [20]:
for j=1:nb_sectors
    @addConstraint(m, sum{poweralloc[j,k], k=1:nb_RBs} <=max_power_per_sector)
end

In [21]:
status = solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:      225
Number of nonzeros in Lagrangian Hessian.............:     1125

Total number of variables............................:      225
                     variables with only lower bounds:      225
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        9
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        9

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [22]:
power_sol=getValue(poweralloc);

In [23]:
sum(power_sol[2,:],2)

1x1 Array{Float64,2}:
 18.8611

In [24]:
power_sol[2,:]

1x25 Array{Float64,2}:
 0.705047  0.930299  0.644469  0.740516  …  0.674128  0.583822  0.70927